# Оценка рыночной стоимости мотоцикла с пробегом

## Выполнили: Болгарин Максим, Моисеенков Павел. BD-11

<br>

**Цель проекта:** проанализировать факторы, влияющие на рыночную стоимость подержанных мотоциклов, и научиться её предсказывать. Какую пользу можно вынести из этого простому человеку? Зная, как образуется стоимость мотоцикла, владелец подобного транспортного средства может подобрать подходящую для него цену для продажи. С другой стороны, будущий владелец сможет оценить адекватность заинтересовавшего его предложения.

**План выполнения:**
1. Поиск данных, формирование датасета
    * Скачивание и парсинг данных.
    * Формирование датасета.
2. Первичный анализ данных
    * Изучение данных: определение смысла признаков, типов данных, наличия пропусков и т.д.
3. Предобработка данных
    * Очистка и отбор данных, выполнение необходимых преобразований.
4. Визуальный анализ данных
    * Поиск скрытых особенностей в данных, анализ корреляций.
5. Применение моделей анализа данных для предсказания стоимости
    * Тестирование нескольких простых моделей, сравнение качества, выбор baseline.
    * Обучение и проверка модели, показывающей наилучшее качество.
6. Вывод

<br>

### 1. Поиск данных, формирование датасета

Источником данных послужил архив объявлений на сайте moto.drom.ru

Всего было обработано около 11000 объявлений из Москвы, Санкт-Петербурга и прочих небольших городов. Скачивание производилось в [другом Jupyter Notebook](https://github.com/maxbolgarin/datamining_project/blob/master/project/parce.ipynb). На выходе мы имеем .csv файл, с ним нам и предстоит работать.

#### Подключим необходимые библиотеки

In [237]:
%matplotlib inline
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt 

import statsmodels.api as sm
from pylab import rcParams

import seaborn as sns

#### Теперь загрузим данные

In [3]:
motorcycles = pd.read_csv('data/motorcycles.csv', index_col='id')

In [4]:
motorcycles.sample(5)

,price,model,mileage,motorcycle_class,year,engine_capacity,engine_strokes,damaged,documents,city,date
id,,,,,,,,,,,
1842,800 000₽,Harley-Davidson Dyna Wide Glide,2 650 км,Чоппер,2011,1 680 куб. см.,2х тактный,Исправен,Есть ПТС,Москва,13 июня 2017
55,420 000₽,Ducati Monster 796,21 000 км,Классика,2013,800 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,18 ноября
4575,300 000₽,Suzuki Boulevard,12 000 км,Чоппер,2007,805 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,17 марта 2016
2026,199 000₽,Kawasaki ZZR 400 2,20 621 км,Спорт-турист,2005,400 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,15 мая 2017
3893,279 000₽,Kawasaki ER-6n,4 367 км,None,2006,600 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,27 июня 2016


In [5]:
motorcycles.head()

,price,model,mileage,motorcycle_class,year,engine_capacity,engine_strokes,damaged,documents,city,date
id,,,,,,,,,,,
1,150 000₽,Honda CRM 250,2 150 км,Эндуро,1992,250 куб. см.,2х тактный,Исправен,Есть ПТС,Москва,30 ноября
2,595 000₽,Suzuki V-Strom DL650A,2 км,Эндуро,2014,645 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,30 ноября
3,159 000₽,Yamaha XVS 400,36 000 км,Чоппер,1997,400 куб. см.,4х тактный,Исправен,Есть ПТС,Краснознаменск,30 ноября
4,98 000₽,Suzuki GN 125,2 677 км,Классика,2008,125 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,30 ноября
5,350 000₽,Yamaha XJ,22 446 км,Спорт-турист,2011,600 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,30 ноября


In [6]:
motorcycles.shape

(11488, 11)

Видим, что данные успешно загрузились и выглядят вполне адекватно. Можем работать дальше.

<br>

### 2. Первичный анализ данных

Посмотрим общую информацию по датасету:

In [7]:
motorcycles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11488 entries, 1 to 11783
Data columns (total 11 columns):
price               11488 non-null object
model               11488 non-null object
mileage             11488 non-null object
motorcycle_class    11488 non-null object
year                11488 non-null object
engine_capacity     11488 non-null object
engine_strokes      11488 non-null object
damaged             11488 non-null object
documents           11488 non-null object
city                11488 non-null object
date                11488 non-null object
dtypes: object(11)
memory usage: 1.1+ MB


In [8]:
motorcycles.nunique()

price               1250
model               3156
mileage             2506
motorcycle_class      16
year                  64
engine_capacity      434
engine_strokes         3
damaged                3
documents              3
city                 116
date                2518
dtype: int64

In [9]:
motorcycles.drop_duplicates().shape

(11390, 11)

Теперь подробнее рассмотрим столбцы:

**price:**

In [10]:
motorcycles.price.value_counts()

150 000₽           240
200 000₽           218
250 000₽           214
300 000₽           192
350 000₽           184
120 000₽           172
180 000₽           164
220 000₽           153
160 000₽           148
230 000₽           147
100 000₽           146
140 000₽           143
130 000₽           142
110 000₽           141
190 000₽           134
Цена не указана    134
170 000₽           130
450 000₽           126
210 000₽           119
280 000₽           118
165 000₽           117
320 000₽           111
90 000₽            110
400 000₽           108
240 000₽           102
80 000₽             96
50 000₽             95
260 000₽            94
135 000₽            90
60 000₽             88
                  ... 
391 900₽             1
221 500₽             1
107 990₽             1
734 900₽             1
51 300₽              1
446 000₽             1
656 000₽             1
154 994₽             1
104 999₽             1
369 999₽             1
618 000₽             1
133 333₽             1
119 500₽   

**model:**

In [11]:
motorcycles.model.value_counts()[:8]

Yamaha YZF R6             162
Honda CBR 600RR           110
Honda CB 400SF            107
Suzuki Boulevard M109R    107
Kawasaki Ninja ZX-6R       91
Yamaha YZF R1              83
Yamaha FZ 6                82
Yamaha                     80
Name: model, dtype: int64

**mileage:**

In [12]:
motorcycles.mileage.value_counts()[:10] / motorcycles.mileage.size

None         0.129439
1 км         0.055710
30 000 км    0.017584
20 000 км    0.016104
25 000 км    0.013057
40 000 км    0.012013
10 000 км    0.011490
35 000 км    0.010794
15 000 км    0.010010
12 000 км    0.009749
Name: mileage, dtype: float64

**motorcycle_class:**

In [13]:
motorcycles.motorcycle_class.value_counts() / motorcycles.motorcycle_class.size

None               0.496431
Чоппер             0.181581
Классика           0.071901
Спортивный         0.062326
Спорт-турист       0.039171
Эндуро             0.034645
Круизер            0.034558
Питбайк            0.022371
Туристический      0.020717
Кроссовый          0.011316
Стритфайтер        0.010097
Кастом             0.009488
Мотард             0.002786
Трайк (трицикл)    0.001306
Детский            0.001219
Триал              0.000087
Name: motorcycle_class, dtype: float64

**year:**

In [14]:
motorcycles.year.value_counts()[:13] / motorcycles.year.size

2014    0.065721
2006    0.061368
2008    0.060498
2007    0.059018
2013    0.054840
2005    0.047180
2003    0.045526
2012    0.043698
2002    0.040477
2004    0.039432
2009    0.038997
2011    0.035864
None    0.034819
Name: year, dtype: float64

**engine_capacity:**

In [15]:
motorcycles.engine_capacity.value_counts()[:14] / motorcycles.engine_capacity.size

400 куб. см.      0.102890
600 куб. см.      0.101671
250 куб. см.      0.083565
1 000 куб. см.    0.049530
650 куб. см.      0.047615
1 300 куб. см.    0.039084
750 куб. см.      0.037169
1 200 куб. см.    0.032382
125 куб. см.      0.028987
1 800 куб. см.    0.027159
1 100 куб. см.    0.025331
800 куб. см.      0.024634
200 куб. см.      0.019760
None              0.019063
Name: engine_capacity, dtype: float64

**engine_strokes:**

In [16]:
motorcycles.engine_strokes.value_counts() / motorcycles.engine_strokes.size

4х тактный    0.841400
None          0.099843
2х тактный    0.058757
Name: engine_strokes, dtype: float64

**damaged:**

In [17]:
motorcycles.damaged.value_counts() / motorcycles.damaged.size

Исправен      0.969185
None          0.017409
Неисправен    0.013405
Name: damaged, dtype: float64

**documents:**

In [18]:
motorcycles.documents.value_counts() / motorcycles.documents.size

Есть ПТС    0.908687
Без ПТС     0.089224
None        0.002089
Name: documents, dtype: float64

**city:**

In [19]:
motorcycles.city.value_counts()[:10] / motorcycles.city.size

Москва             0.597580
Санкт-Петербург    0.270021
Серпухов           0.009053
Подольск           0.004788
Мытищи             0.004526
Жуковский          0.003743
Одинцово           0.003743
Дмитров            0.003395
Зеленоград         0.003221
Егорьевск          0.003134
Name: city, dtype: float64

**date:**

In [20]:
motorcycles.sample(10).date

id
10467         1 июня 2015
4583        15 марта 2016
8595            29 ноября
5857       28 апреля 2015
2595       22 апреля 2017
8461      14 августа 2009
9432       17 ноября 2017
2062          12 мая 2017
11464      5 декабря 2012
3319     27 сентября 2016
Name: date, dtype: object

Пройдемся по столбцам и, на основе уже имеющейся информации, дадим им некоторое описание:
1. **id:** просто порядковый номер мотоцикла, который вряд ли несет какую либо полезную информацию.
2. **price:** цена мотоцикла, целевая переменная. Видим, что есть объекты со значением "Цена не указана". Их нам придется отбросить. Также можно заметить, что не у всех мотоциклов стоит адекватная цена (290₽ и т.п.). Такие объекты тоже скорее всего можно отбросить, так как они вряд ли составляют весомую долю от всех остальных. Также следует преобразовать стоимость в числовой тип, отбросив символ "₽".
3. **model:** модель мотоцикла. Можно построить отдельный столбец **manufacturer**, в котором указывать только производителя, и рассматривать его как новое свойство.
4. **mileage:** пробег мотоцикла. 13% имеют пропуски, возможно это из за того, что такие мотоциклы являются новыми. Было бы странно, если бы продавец не указывал хотя бы примерный пробег своего подержанного транспортного средства, ведь это является один из самых главных факторов при покупке. Так что заполним пропуски нулями. Также 5.6% имеют пробег "1 км", это странное значение можно тоже занулить. Значения стоит привести к числовым, отбросить подстроку "км".
5. **motorcycle_class:** класс мотоцикла. Как видим, около 50% всех объектов не имеют информции о классе. Возможно, этот столбец придется убрать из рассмотрения, так как заполнить пропуски сложно без искажения реальной информации. Считать отсутстиве значения как свойство не имеет смысла, так как класс присущ каждому мотоциклу, и отстуствие информации об этом характеризует скорее не мотоцикл, а человека, составляющего объявление.
6. **year:** год выпуска мотоцикла. Как видим, значения отсутсвуют лишь у 3.5% всех объявлений, что не значительно.
7. **engine_capacity:** объем двигателя. Пропуски имеют 2% объектов. Значения можно преобразовать с числовой тип, отбросив подстроку "куб. см.".
8. **engine_strokes:** число тактов двигателя. Видим, что всего 2 разных значения, и что больше 80% мотоциклов обладают четырехтактным двигателем. Также имеем 10% пропусков. Можно попробовать заменить пропуски на "4х тактный", так как большинство двигателей именно такие, и скорее всего продавец скорее всего знал бы, что у него особенный "2х тактный" двигатель, и указал бы это. Не думаю, что ошибка будет более 1% от общего количества. Также обрежем строку до одного числа, 2 или 4.
9. **damaged:** состояние мотоцикла. Пропуски имеют 2%, и с большой уверенностью можно заполнить их значениями "Исправен", так как предполагается, что если мотоцикл сломан, то продавец об этом скорее всего сообщит заранее (если он добросовестный, конечно). Также значение "Исправен" можно закодировать под 0, а "Неисправен" - под 1.
10. **documents:** наличие ПТС у продавца. Аналогично с прошлым столбцом, можно заполнить пропуски самым популярным вариантом и закодировать "Есть ПТС" как 0 и "Без ПТС" как 1.
11. **city:** город, в котором продается мотоцикл. Пропусков нет.
12. **date:** как можно заметить из семпла, у некоторых объявлений есть информация о годе, а у некоторых нет. Те, у которых нет - объявления за 2018 год. Надо подправить это и дописать 2018 к этим датам.

<br>

### 3. Предобработка данных

Для начала избавимся от повторяющихся значений и запомним изначальное число объявлений:

In [21]:
motorcycles = motorcycles.drop_duplicates()
number_of_motocycles = motorcycles.shape[0]

Напишем функцию drop_none для упрощения работы:

In [22]:
def drop_none(df):
    return df.replace(to_replace='None', value=np.nan).dropna()

Теперь произведем обработку слобцов в соответствии с замечаниями, данными выше.

Начнем с преобразования целевого столбца **price**:

In [23]:
motorcycles.price = drop_none(motorcycles.price)
motorcycles = motorcycles[motorcycles.price != "Цена не указана"]
motorcycles.price = motorcycles.price.str[:-1].replace(' ', '', regex=True).astype(np.int64)
motorcycles.price = motorcycles.price[motorcycles.price.values > 4999]

Перейдем к столбцу **model**:

In [24]:
(motorcycles.model.shape[0] - drop_none(motorcycles.model).shape[0]) / motorcycles.model.shape[0]

0.0010660033756773562

Как видим, None имеют один процент мотоциклов, поэтому отбросим их и создадим новый столбец **manufacturer**:

In [25]:
motorcycles.model = drop_none(motorcycles.model)
motorcycles['manufacturer'] = [item[0] if type(item) is list else item
                               for item in motorcycles.model.str.split(' ').values]

Преобразуем столбец **mileage**:

In [26]:
motorcycles.mileage = motorcycles.mileage.str.replace("None", "0")
motorcycles.mileage = motorcycles.mileage.str.replace("1 км", "0", regex=True)
motorcycles.mileage = motorcycles.mileage.str.replace("км", "", regex=True)
motorcycles.mileage = motorcycles.mileage.str.replace(' ', '', regex=True).astype(np.int64)

Как решили ранее, с **motorcycle_class** работать сложно, поэтому просто не будем рассматривать этот столбец.

In [27]:
motorcycles = motorcycles.drop(['motorcycle_class'], axis=1)

Отбросим пропуски и преобразуем тип значений столбца **year** в integer:

In [28]:
motorcycles.year = drop_none(motorcycles.year)
motorcycles.year = motorcycles.year.dropna().astype(np.int32)

На очереди **engine_capacity**:

In [29]:
motorcycles.engine_capacity = drop_none(motorcycles.engine_capacity)
motorcycles.engine_capacity = motorcycles.engine_capacity.str.replace("куб. см.", '', regex=True)
motorcycles.engine_capacity = motorcycles.engine_capacity.str.replace(' ', '', regex=True)
motorcycles.engine_capacity = motorcycles.engine_capacity.dropna().astype(np.int64)

В столбце **engine_strokes** заменим None на "4х тактный":

In [30]:
motorcycles.engine_strokes = motorcycles.engine_strokes.str.replace("None", "4", regex=True)
motorcycles.engine_strokes = motorcycles.engine_strokes.str.replace("х тактный", '', regex=True)
motorcycles.engine_strokes = motorcycles.engine_strokes.dropna().astype(np.int64)

Преобразуем **damaged**:

In [31]:
motorcycles.damaged = motorcycles.damaged.str.replace("None", "0", regex=True)
motorcycles.damaged = motorcycles.damaged.str.replace("Исправен", "0", regex=True)
motorcycles.damaged = motorcycles.damaged.str.replace("Неисправен", "1", regex=True)
motorcycles.damaged = motorcycles.damaged.dropna().astype(np.int64)

Аналогично **documents**:

In [32]:
motorcycles.documents = motorcycles.documents.str.replace("None", "0", regex=True)
motorcycles.documents = motorcycles.documents.str.replace("Есть ПТС", "0", regex=True)
motorcycles.documents = motorcycles.documents.str.replace("Без ПТС", "1", regex=True)
motorcycles.documents = motorcycles.documents.dropna().astype(np.int64)

Наконец, преобразуем столбец **date**. Сначала добавим год тем объектам, у которых его нет:

In [33]:
motorcycles.date = [s + " 2018" if len(s.split()) == 2 else s for s in motorcycles.date.values]

По хорошему, строки с датой надо бы привести к формату datetime. Однако, не совсем понятно, как работать с названиями месяцев на русском языке. Поэтому поменяем их на английский, а потом приведем значения к datetime:

In [34]:
rus_eng_month = {
    'января': 'January',
    'февраля': 'February',
    'марта': 'March',
    'апреля': 'April',
    'мая': 'May',
    'июня': 'June',
    'июля': 'July',
    'августа': 'August',
    'сентября': 'September',
    'октября': 'October',
    'ноября': 'November',
    'декабря': 'December',  
}

dates = []
for s in motorcycles.date.values:
    temp_s = s.split()
    dates.append(temp_s[0] + ' ' + rus_eng_month[temp_s[1]] + ' ' + temp_s[2])
    
motorcycles.date = [datetime.strptime(s, '%d %B %Y').strftime('%Y-%m-%d') for s in dates]
motorcycles.date = pd.to_datetime(motorcycles.date.dropna()).dt.date

Отбросим оставшиеся неопределенные значения:

In [35]:
motorcycles = drop_none(motorcycles)
motorcycles = motorcycles.dropna()

Вот так теперь выглядит таблица:

In [36]:
motorcycles.sample(5)

,price,model,mileage,year,engine_capacity,engine_strokes,damaged,documents,city,date,manufacturer
id,,,,,,,,,,,
261,700000.0,Kawasaki Ninja 1000,6000,2016.0,1043.0,4,0,0,Москва,2018-08-23,Kawasaki
3816,205000.0,Yamaha R1,33500,2001.0,998.0,4,0,0,Электросталь,2016-07-10,Yamaha
11712,20000.0,flamingo,0,2007.0,50.0,4,0,0,Санкт-Петербург,2009-10-27,flamingo
4889,430000.0,Big Dog Mastif,39000,2003.0,1700.0,4,0,0,Москва,2015-12-03,Big
206,45000.0,Урал М-67 36,1111,1989.0,650.0,4,0,0,Москва,2018-09-08,Урал


In [37]:
motorcycles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10719 entries, 1 to 11783
Data columns (total 11 columns):
price              10719 non-null float64
model              10719 non-null object
mileage            10719 non-null int64
year               10719 non-null float64
engine_capacity    10719 non-null float64
engine_strokes     10719 non-null int64
damaged            10719 non-null int64
documents          10719 non-null int64
city               10719 non-null object
date               10719 non-null object
manufacturer       10719 non-null object
dtypes: float64(3), int64(4), object(4)
memory usage: 1004.9+ KB


Посмотрим, сколько осталось объектов после всех преобразований:

In [38]:
print('Было: {}'.format(number_of_motocycles))
print('Осталось: {}'.format(motorcycles.shape[0]))
print('Датасет уменьшился на {:.3}% от первоначального размера'.format(
    100*(number_of_motocycles - motorcycles.shape[0]) / number_of_motocycles)
     )

Было: 11390
Осталось: 10719
Датасет уменьшился на 5.89% от первоначального размера


Как видим, после данных преобразований датасет уменьшился незначительно. Теперь обратим внимание на стобец **manufacturer**.

In [39]:
motorcycles.manufacturer.value_counts()

Honda                 2483
Yamaha                1873
Suzuki                1379
Kawasaki              1016
BMW                    436
Harley-Davidson        412
KTM                    196
Ducati                 164
Irbis                  121
Stels                  119
HONDA                  101
Racer                   87
Kayo                    87
YAMAHA                  86
Triumph                 79
Урал                    74
Aprilia                 67
Baltmotors              52
Иж                      47
Ява                     45
SUZUKI                  36
BSE                     35
Victory                 33
Sym                     31
NO.                     28
Минск                   28
Днепр                   27
ABM                     26
CBR                     22
Wels                    21
                      ... 
PC800                    1
ER-6n                    1
B-King                   1
GAS                      1
Black                    1
intruder                 1
X

Как можно заметить, встречаются одинаковые названия производителей, но написанные разным способом. Компьютер их различает как разных производителей, нам же надо преобразовать их к одному единому виду.

Если "NO.", то название производителя начинается с третьего слова в **model**:

In [40]:
motorcycles.loc[motorcycles.manufacturer == "NO.", "manufacturer"] = \
            [item[2] for item in motorcycles[motorcycles.manufacturer == "NO."].model.str.split().values]

Honda $\leftarrow$ HONDA, honda, ХОНДА, Хонда, CBR, CB, CB400

In [41]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("HONDA", "Honda", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("ХОНДА", "Honda", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("Хонда", "Honda", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("honda", "Honda", regex=True)
motorcycles.loc[motorcycles.manufacturer == "CBR", "manufacturer"] = "Honda"
motorcycles.loc[motorcycles.manufacturer == "CB", "manufacturer"] = "Honda"
motorcycles.loc[motorcycles.manufacturer == "CB400", "manufacturer"] = "Honda"
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("CBR", "Honda", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("CB", "Honda", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("CB400", "Honda", regex=True)

Yamaha $\leftarrow$ YAMAHA, yamaha

In [42]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("YAMAHA", "Yamaha", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("yamaha", "Yamaha", regex=True)

Harley-Davidson $\leftarrow$ Harleu-Davidson, HARLEY-DAVIDSON, Харли, Harley, Harley-Davidson, Harley-Davidson

In [43]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("Harleu-Davidson", "Harley-Davidson", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("HARLEY-DAVIDSON", "Harley-Davidson", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("Харли", "Harley-Davidson", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("Harley-Davidson", "Harley-Davidson", regex=True)
motorcycles.loc[motorcycles.manufacturer == "Harley", "manufacturer"] = "Harley-Davidson"

Kawasaki $\leftarrow$ KAWASAKI, КАВАСАКИ, Кавасаки, кавасаки, kawasaki

In [44]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("KAWASAKI", "Kawasaki", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("КАВАСАКИ", "Kawasaki", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("Кавасаки", "Kawasaki", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("кавасаки", "Kawasaki", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("kawasaki", "Kawasaki", regex=True)

Suzuki $\leftarrow$ сузуки, suzuki, SUZUKI, Сузуки

In [45]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("сузуки", "Suzuki", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("suzuki", "Suzuki", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("Сузуки", "Suzuki", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("SUZUKI", "Suzuki", regex=True)

Ява $\leftarrow$ ЯВА

In [46]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("ЯВА", "Ява", regex=True)

Иж $\leftarrow$ ИЖ, иж, ИЖ-49, Иж-49

In [47]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("ИЖ", "Иж", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("иж", "Иж", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("ИЖ-49", "Иж", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("Иж-49", "Иж", regex=True)

CFMOTO $\leftarrow$ Cfmoto

In [48]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("Cfmoto", "CFMOTO", regex=True)

Минск $\leftarrow$ минск

In [49]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("минск", "Минск", regex=True)

Triumph $\leftarrow$ triumph, TRIUMPH

In [50]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("triumph", "Triumph", regex=True)
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("TRIUMPH", "Triumph", regex=True)

Ducati $\leftarrow$ DUCATI

In [51]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("DUCATI", "Ducati", regex=True)

Dio $\leftarrow$ dio

In [52]:
motorcycles.manufacturer = motorcycles.manufacturer.str.replace("dio", "Dio", regex=True)

Теперь самым редким производителям дадим метку Other:

In [56]:
names = [motorcycles.manufacturer.value_counts().index[i] 
         if m < 10 else None for i, m in enumerate(motorcycles.manufacturer.value_counts())]
names.append("Мотоцикл")
names.append("Кроссовый")
names.append("Moto")
names = np.array(names)
names = names[names != np.array(None)]

for moto in motorcycles.manufacturer.values:
    if moto in names:
        motorcycles.loc[motorcycles.manufacturer == moto, "manufacturer"] = "Other"
        
motorcycles.manufacturer.value_counts()

49

In [228]:
motorcycles.manufacturer.value_counts()

Honda              1452
Yamaha             1152
Suzuki              779
Kawasaki            655
Other               430
Harley-Davidson     320
BMW                 300
KTM                 143
Ducati              126
Stels                66
Triumph              60
Racer                57
Irbis                50
Kayo                 48
Урал                 44
Aprilia              37
NO.                  28
Baltmotors           28
Ява                  24
Sym                  23
BSE                  21
Иж                   21
CBR                  17
BRP                  17
Hyosung              16
Минск                16
CB                   15
Husqvarna            14
YCF                  14
Dio                  13
                   ... 
FAt                   1
Такт-30               1
x4                    1
FZ1000S               1
CB600                 1
CBR1100               1
Camp                  1
ns50R                 1
vn800                 1
diversion             1
FZ400           

Теперь у нас есть неплохая информация о производителях. Можно отбросить столбец с моделями, так как нам не нужна подобная излишняя информация:

In [54]:
motorcycles = motorcycles.drop(["model"], axis=1)

Осталось сделать некоторые преобразования со столбцом **city**:

In [58]:
cities = [motorcycles.city.value_counts().index[i] 
         if m < 10 else None for i, m in enumerate(motorcycles.city.value_counts())]

cities = np.array(cities)
cities = cities[cities != np.array(None)]

for city in motorcycles.city.values:
    if city in cities:
        motorcycles.loc[motorcycles.city == city, "city"] = "Другой"
        
motorcycles.city.value_counts()

Москва              6353
Санкт-Петербург     2957
Other                211
Серпухов             101
Подольск              52
Мытищи                47
Жуковский             42
Одинцово              40
Дмитров               36
Егорьевск             36
Пушкино               35
Зеленоград            34
Ногинск               32
Коломна               32
Раменское             30
Домодедово            30
Щелково               30
Ступино               29
Воскресенск           28
Дубна                 27
Балашиха              27
Химки                 27
Сергиев Посад         26
Люберцы               26
Орехово-Зуево         25
Наро-Фоминск          25
Королев               25
Железнодорожный       24
Лобня                 24
Луховицы              23
Истра                 21
Солнечногорск         20
Ивантеевка            20
Электросталь          18
Клин                  17
Волоколамск           16
Красногорск           16
Долгопрудный          13
Видное                12
Чехов                 12


Предварительная обработка данных закончена. Теперь можем приступать к анализу датасета и построению моделей.

In [227]:
motorcycles.sample(5)

,price,model,mileage,year,engine_capacity,engine_strokes,damaged,documents,date,manufacturer
id,,,,,,,,,,
1106,480000.0,Honda Fury,12000,2011.0,1300.0,4,0,0,2017-09-07,Honda
5272,130000.0,Suzuki GSX R750,26500,2007.0,750.0,4,0,1,2014-08-27,Suzuki
886,130000.0,Honda Steed,56000,1998.0,400.0,4,0,0,2017-12-07,Honda
3403,120000.0,Yamaha TT-R 125l,0,2003.0,129.0,4,0,0,2016-05-29,Yamaha
2459,120000.0,Honda CB 400SF,35000,1993.0,1993.0,2,0,0,2016-12-09,Honda


<br>

### 4. Визуальный анализ данных

Проанализируем данные визуально. Построим несколько графиков и попробуем вынести из них полезную информацию.

# 5. 

Выбираем признаки

In [253]:
x_labels = ['engine_capacity', 'mileage', 'year', 'damaged','documents']
X = motorcycles.loc[:, x_labels]

Выделяем целевую переменную — она записана в столбце price.

In [254]:
Y = motorcycles['price']

In [255]:
from sklearn.tree import DecisionTreeClassifier

Обучаем решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию.

In [258]:
clf = DecisionTreeClassifier(random_state=241)
clf=clf.fit(X,Y)  # Обучение модели производится с помощью функции fit(X,Y).

Вычисляем важность признаков и найдём признаки с наибольшей важностью.

In [259]:
importances = pd.Series(clf.feature_importances_, index=x_labels)    # обученный классификатор - clf.feature_importances_
#print(' '.join(importances.sort_values(ascending=False).head(4).index.values))
imp=importances.sort_values(ascending=False).head(5).index.values  
print(' '.join(imp))

mileage year engine_capacity documents damaged
